In [1]:
import inventory
import numpy as np
import pandas as pd
import pykonal
import scipy.optimize

In [2]:
EVENTS = pd.read_hdf("../data/catalog.hdf5", key="events")
ARRIVALS = pd.read_hdf("../data/catalog.hdf5", key="arrivals")
ARRIVALS["location"] = ""
ARRIVALS["handle"] = ARRIVALS["network"] + "." + ARRIVALS["station"] + "." + ARRIVALS["location"] + "." + ARRIVALS["phase"]

In [116]:
class EQLocator(object):

    def __init__(
        self, 
        tt_inventory,
        delta=np.array([
            20,
            np.radians(0.2),
            np.radians(0.2),
            10
        ])
    ):
        self._tti = inventory.TTInventory(tt_inventory)
        self.delta = delta


    def __exit__(self):
        self.__del__()


    def __del__(self):
        self.tti.f5.close()


    @property
    def arrivals(self):
        return (self._arrivals)
    
    @arrivals.setter
    def arrivals(self, value):
        self._arrivals = value
        self._bounds = None
        self._tt = {handle:
            self._tti.read(
                handle,
                min_coords=self.bounds[0, :-1],
                max_coords=self.bounds[1, :-1]
            )
            for handle in value["handle"]
       }
        
    @property
    def bootstrap_arrivals(self):
        return (self._bootstrap_arrivals)
    
    @bootstrap_arrivals.setter
    def bootstrap_arrivals(self, value):
        self._bootstrap_arrivals = value


    @property
    def bounds(self):
        if not hasattr(self, "_bounds") or self._bounds is None:
            loc = self.grid_search()
            self._bounds = np.array([loc-self.delta, loc+self.delta])
        return (self._bounds)


    @property
    def delta(self):
        """
        Search range around initial location.
        """
        return (self._delta)


    @delta.setter
    def delta(self, value):
        self._delta = value
        
            
    @property
    def tti(self):
        return (self._tti)


    def basinhopping(self, order=None, **kwargs):
        return(
            scipy.optimize.basinhopping(
                lambda coords: self.norm(coords, order=order),
                self.grid_search(),
                **kwargs
            )
        )

    
    def bootstrap_sample(self, coords):
        
        arrivals = self.arrivals.copy()
        r = self.residuals(coords, bootstrap=False)
        arrivals["time"] += np.random.choice(r, size=len(r))
        
        self.bootstrap_arrivals = arrivals

    
    def differential_evolution(self, order=None, bootstrap=False, **kwargs):
        return(
            scipy.optimize.differential_evolution(
                lambda coords: self.norm(coords, order=order, bootstrap=bootstrap),
                bounds=self.bounds.T,
                **kwargs
            )
        )


    def dual_annealing(self, order=None, **kwargs):
        return(
            scipy.optimize.dual_annealing(
                lambda coords: self.norm(coords, order=order),
                bounds=self.bounds.T,
                **kwargs
            )
        )


    def grid_search(self):
        """
        Return location estimate from grid search.
        """
        t0 = np.array([
            arrival["time"] - self.tti.read(arrival["handle"]).values
            for _,  arrival in self.arrivals.iterrows()
        ])
        std = np.std(t0, axis=0)
        idx_min = np.unravel_index(np.argmin(std), std.shape)
        t0 = np.mean(t0, axis=0)[idx_min]
        loc = np.array([*self.tti.nodes[idx_min], t0])

        return (loc)
    
    def locate(self, method="differential evolution", order=None, **kwargs):
        
        if method.lower() == "basin hopping":
            return (self.basinhopping(order=order, **kwargs))
        
        elif method.lower() == "differential evolution":
            return (self.differential_evolution(order=order, **kwargs))
        
        elif method.lower() == "dual annealing":
            return (self.dual_annealing(order=order, **kwargs))
        
        elif method.lower() == "grid search":
            return (self.grid_search())
    
        elif method.lower() == "shgo":
            return (self.shgo(order=order, **kwargs))

    
    def norm(self, coords, order=None, bootstrap=False):
        return (
            np.linalg.norm(
                self.residuals(coords, bootstrap=bootstrap), 
                ord=order
            )
        )

    
    def residuals(self, coords, bootstrap=False):
        if bootstrap is False:
            arrivals = self.arrivals
        elif bootstrap is True:
            arrivals = self.bootstrap_arrivals
        else:
            raise (ValueError)
            
        tt = np.array([
            self._tt[
                handle
            ].value(
                coords[:3], 
                null=np.inf
            )
            for handle in arrivals["handle"]
        ])
        
        return (arrivals["time"].values - coords[3] - tt)    
    
    def shgo(self, order=None, **kwargs):
        return(
            scipy.optimize.shgo(
                lambda coords: self.norm(coords, order=order),
                bounds=self.bounds.T,
                **kwargs
            )
        )

locator = EQLocator("../data/tts.hdf5")

In [117]:
%%time
event = EVENTS.iloc[1]
locator.arrivals = ARRIVALS.set_index("event_id").loc[event["event_id"]]

CPU times: user 259 ms, sys: 149 ms, total: 408 ms
Wall time: 405 ms


In [118]:
%%time
loc0 = locator.differential_evolution(order=2, bootstrap=False)
loc0.x

CPU times: user 315 ms, sys: 12.4 ms, total: 328 ms
Wall time: 317 ms


array([ 6.35897424e+03,  9.84568129e-01, -2.03574312e+00,  1.19914605e+09])

In [131]:
%%time
boots = np.empty((0, 4))
for i in range(100):
    locator.bootstrap_sample(loc0.x)
    loc = locator.differential_evolution(order=2, bootstrap=True)
    boots = np.vstack([boots, loc.x])

CPU times: user 1min 2s, sys: 182 ms, total: 1min 2s
Wall time: 1min 2s
